In [ ]:
# Import libraries, models, utilities, and dataset loader
from models.cnn_lstm import CNN_LSTM
from models.cnn import CNN
from models.lstm import LSTM
from models.mlp import MLP
from utils import load_model, DATA_DIR, BATCH_SIZE, NUM_WORKER
from loader.fi_loader import FIDataset

import torch
from torch.utils.data import DataLoader

import os
from itertools import product
from tqdm import tqdm

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Generate Testing Statistics

In [2]:
testing_results = {
    'model_type': [],
    'train_data_size': [],
    'prediction_horizon': [],
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1': []
}

In [ ]:
# Load trained model weights from disk
testing_params = product(zip(['CNN_LSTM', 'CNN', 'LSTM', 'MLP'], [CNN_LSTM, CNN, LSTM, MLP]), [1, 3, 5, 8], [0, 2, 4])

for (model_name, model_type), cf, k in tqdm(testing_params):
    model_path = os.path.join('.', 'trained_models', model_name)
    trained_model_path = os.path.join(model_path, f'{model_name}_Zscore_CF{cf}_pred_{k}.pth')
    print(trained_model_path)

    trained_model = load_model(model_type, trained_model_path)

    # Load validation/test dataset
    test_data = FIDataset(DATA_DIR, 'Zscore', cf, k=k, train=False)
    test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKER)

    trained_model.eval()
    trained_model.to(trained_model.device)

    all_predictions = []
    all_targets = []

    # Predict labels and calculate accuracy
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(trained_model.device, dtype=torch.float32), targets.to(trained_model.device, dtype=torch.int64)
            outputs = trained_model(inputs)
            _, predictions = torch.max(outputs, 1)

            all_predictions.extend(predictions.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    accuracy = accuracy_score(all_targets, all_predictions)
    precision = precision_score(all_targets, all_predictions, average='weighted', zero_division=0)
    recall = recall_score(all_targets, all_predictions, average='weighted', zero_division=0)
    f1 = f1_score(all_targets, all_predictions, average='weighted', zero_division=0)

    testing_results['model_type'].append(model_name)
    testing_results['train_data_size'].append(cf)
    testing_results['prediction_horizon'].append(k)
    testing_results['accuracy'].append(accuracy)
    testing_results['precision'].append(precision)
    testing_results['recall'].append(recall)
    testing_results['f1'].append(f1)

0it [00:00, ?it/s]

.\trained_models\CNN_LSTM\CNN_LSTM_Zscore_CF1_pred_0.pth


1it [00:14, 14.83s/it]

.\trained_models\CNN_LSTM\CNN_LSTM_Zscore_CF1_pred_2.pth


2it [00:28, 14.08s/it]

...

46it [09:04, 11.81s/it]

.\trained_models\MLP\MLP_Zscore_CF8_pred_2.pth


47it [09:17, 12.34s/it]

.\trained_models\MLP\MLP_Zscore_CF8_pred_4.pth


48it [09:31, 11.91s/it]


# Save Testing Stats

In [4]:
import pandas as pd
import pickle

testing_results_df = pd.DataFrame(testing_results)
with open('testing_results.pkl', 'wb') as f:
    pickle.dump(testing_results_df, f)

In [5]:
testing_results_df

,model_type,train_data_size,prediction_horizon,accuracy,precision,recall,f1
0,CNN_LSTM,1,0,0.573875,0.484466,0.573875,0.465098
1,CNN_LSTM,1,2,0.374123,0.374969,0.374123,0.374240
2,CNN_LSTM,1,4,0.515698,0.500366,0.515698,0.496513
3,CNN_LSTM,3,0,0.603154,0.534757,0.603154,0.478451
4,CNN_LSTM,3,2,0.634582,0.633881,0.634582,0.624495
5,CNN_LSTM,3,4,0.662478,0.662496,0.662478,0.662375
6,CNN_LSTM,5,0,0.616551,0.537847,0.616551,0.506248
7,CNN_LSTM,5,2,0.685697,0.691451,0.685697,0.675131
8,CNN_LSTM,5,4,0.711747,0.714468,0.711747,0.711004
9,CNN_LSTM,8,0,0.816011,0.808944,0.816011,0.795133
